In [1]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

In [2]:
from diorep import contrast,reduceNoise
from diorep.arrman import compute_db
from diorep.arrman import get_image
from diorep.arrman import padSample
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
import IPython.display as ipd
import random
import matplotlib.pyplot as plt
%matplotlib inline
import random
import math
import decimal
from diorep.arrplot import plotSignal
from diorep.fmanip import clip_category,gen_noise_file,get_amplitude
from diorep.arrman import lenArray
from diorep.reduceNoise import retNoiseClip,reduceNoiseNR
from diorep. fmanip import assign_probability
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import math
import librosa
from diorep.arrman import get_melspectr,random_power,mono_to_color,normalize
random.seed(123)
from diorep.arrman import get_wav_transforms
from diorep.fmanip import generate_bkgnoise
import os
from diorep.fmanip import silence_removal

C:\Users\hamhamm\anaconda3\envs\tf_gpu\lib\site-packages\noisereduce\noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
train=pd.read_csv('train1.csv')


In [4]:
train.head()

,fn,label,Sample_Length,Noise_fn,Noise_Amplitude,dB_Noise_SPL,Clip_Group,Prob_Accuracy,Audio_amplitude,dB_Audio_SPL
0,PadSampleAudio/IV38R7F.wav,akawuka,54243,Noise_clip/IV38R7F.wav,0.008996,35,best,1.0,0.721681,73
1,PadSampleAudio/KM4SKWT.wav,banana,55261,Noise_clip/KM4SKWT.wav,0.356834,67,bad,0.2,0.966937,75
2,PadSampleAudio/F5POSU9.wav,obulwadde,55069,Noise_clip/F5POSU9.wav,0.782890,74,bad,0.2,1.052848,76
3,PadSampleAudio/MMVDXG2.wav,nnyaanya,54243,Noise_clip/MMVDXG2.wav,0.015667,40,good,0.7,0.442655,69
4,PadSampleAudio/9TVM96F.wav,pampu,55566,Noise_clip/9TVM96F.wav,0.743068,73,bad,0.2,0.765559,73


In [5]:
class HyperParameters():
    def __init__(self,bias=4,n_clip_length=500,audibility_threshold=50):
        """
        PARAMETERS
        ----------
        bias: Bias parameter for the Noise Threshold
        n_clip_length: Noise Clip Length
        """
        self.bias=bias
        self.sig_1p='audio_files/01RD8LR.wav'
        self.sig_2p='audio_files/0259OO7.wav'
        self.bkg_N='audio_files/SMNDURA.wav'
        self.sr=22050
        self.amp_1N=0.007746608
        self.amp_2N_ref=0.00015370044
        self.noise_threshold=compute_db(self.amp_1N,self.amp_2N_ref) + self.bias
        self.NOISE_DIR='Noise_clip'
        self.n_clip_length=n_clip_length
        self.audibility_threshold=audibility_threshold
        self.bkg_N_path='bkg_Noise'


class HyperParamsTransform():
    """
    Class to define the Hyper-parameters
    """
    def __init__(self,n_mels,img_width,padSamplePath='padSampleDir',sig_len_max=60858):
        """
        PARAMETERS
        ----------
        n_mels: n_mels
        hop_length: hop_length
        INIT
        ----
        img_width
        n_mels
        hop_length
        padSamplePath: path to save padded input
        """
        self.n_mels=n_mels
        self.img_width=img_width 
        self.sig_len_max=sig_len_max
        self.hop_length=math.ceil(self.sig_len_max/self.img_width)
        self.sr=22050
        self.div_coef=100
        self.padSamplePath=padSamplePath
        

In [6]:
hp=HyperParameters()
hpT=HyperParamsTransform(224,224,'padSampleDir_shift200')

In [7]:
# print the noise Threshold
print(f"The Noise Threshold in Decibel is: {hp.noise_threshold} dB ")

The Noise Threshold in Decibel is: 38 dB 


In [8]:
train['Sample_Length']=train.fn.transform(lambda x: lenArray(x) )

In [9]:
train.head()

,fn,label,Sample_Length,Noise_fn,Noise_Amplitude,dB_Noise_SPL,Clip_Group,Prob_Accuracy,Audio_amplitude,dB_Audio_SPL
0,PadSampleAudio/IV38R7F.wav,akawuka,60858,Noise_clip/IV38R7F.wav,0.008996,35,best,1.0,0.721681,73
1,PadSampleAudio/KM4SKWT.wav,banana,60858,Noise_clip/KM4SKWT.wav,0.356834,67,bad,0.2,0.966937,75
2,PadSampleAudio/F5POSU9.wav,obulwadde,60858,Noise_clip/F5POSU9.wav,0.782890,74,bad,0.2,1.052848,76
3,PadSampleAudio/MMVDXG2.wav,nnyaanya,60858,Noise_clip/MMVDXG2.wav,0.015667,40,good,0.7,0.442655,69
4,PadSampleAudio/9TVM96F.wav,pampu,60858,Noise_clip/9TVM96F.wav,0.743068,73,bad,0.2,0.765559,73


# PadSample

In [10]:
from diorep.arrman import padSample

In [11]:
def pad(train,hpT):
    if not os.path.exists(hpT.padSamplePath):
        os.makedirs(hpT.padSamplePath)
    for i in range(len(train)):
        _=padSample(train.Sample_Length.loc[i],train.Sample_Length.max(),train.fn.loc[i]) 
        fname=train.fn.loc[i].split('/')
        fname=fname[1]
        fname=hpT.padSamplePath + '/' + fname

        wav.write(fname,hp.sr,_)
    

In [12]:
pad(train,hpT)

# Generating MelSpectrogram

In [13]:
from diorep.arrman import mono_to_color,get_image,get_melspectr,normalize
import librosa.display
from diorep.saug import pitchShift,shiftTime,timeStretch
from diorep.reduceNoise import reduceNoiseNR

# reduce Noise

# shiftTime

In [14]:
import random 
random.seed(42)

In [ ]:
image_Path='shift_time_mel_200'
time=200

mel_=[]    

if not os.path.exists(image_Path[j]):
    os.makedirs(image_Path[j])
for i in range(911,1109):
    _,arr=wav.read(train.fn.loc[i])
    a=reduceNoiseNR(hp.sr,arr,500)
    a=shiftTime(a,time)
    a=get_image(a,hpT)
    a=mono_to_color(a)
    a=normalize(a)
    img=plt.imshow(a[0])
    fname=train.fn.loc[i].split('/')
    fname=fname[1]
    fname=fname.split('.')
    fname=fname[0]
    fname=image_Path[j] + '/' + fname + '.png'
    mel_.append(fname)
    plt.savefig(fname)